In [1]:
#files
import sys
#from os import listdir
sys.path.append('../data')
sys.path.append('../Librerias')
#from os.path import isfile, join

#own libs
import utils
import caract as dc
import dataset as ds
from models import SimpleRNN, LSTM
#from lstm_models import LSTM
import utils_2 as ut2
from traindata import trainData



#data and visualization
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR

sns.set_style("darkgrid")


import numpy as np
from tqdm import tqdm



np.random.seed(seed=77)

## **data**

In [2]:
#fpath = "/Users/consu/OneDrive/Documentos/GitHub/Tesis/data/low/"
fpath = "/Users/granosftp/Documents/GitHub/Tesis/data/low/"
fname = "file1_080124.mat"
#cutoff = [8/1000, 11/1000]

data = trainData(fpath, fname, 'Amplitude')
train, val, test = data.split_data('Amplitude')

In [3]:
# sub sampleo

train = utils.subsample(train, 2)
val = utils.subsample(val, 2)
test = utils.subsample(test, 2)

In [4]:
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train.reshape(-1, 1))
val_scaled = scaler.transform(val.reshape(-1, 1))
test_scaled = scaler.transform(test.reshape(-1, 1))

In [5]:
x_train, y_train = ut2.create_sequences(train_scaled[:10000], 1000, 1)
x_train = x_train.squeeze()
y_train = y_train.squeeze()

x_val, y_val = ut2.create_sequences(val_scaled[:5000], 1000, 1)
x_val = x_val.squeeze()
y_val = y_val.squeeze()

x_test, y_test = ut2.create_sequences(test_scaled[:3000], 1000, 1)
x_test = x_test.squeeze()
y_test = y_test.squeeze()

x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape

((9000, 1000), (9000,), (4000, 1000), (4000,), (2000, 1000), (2000,))

## **modelo svr**

In [6]:
from sklearn.model_selection import RepeatedKFold, GridSearchCV, cross_validate
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.pipeline import Pipeline
 

In [7]:
from sklearn.model_selection import PredefinedSplit

model = SVR()
param = {'kernel' : ('linear', 'poly', 'rbf', 'sigmoid'),
         'C' : [1,5,10],
         'degree' : [3,8],
         'coef0' : [0.01,10,0.5],
         'gamma' : ('auto','scale')}

#array para el predefinedsplit
test_fold = [-1 for _ in range(x_train.shape[0])] + [0 for _ in range(x_val.shape[0])]
ps = PredefinedSplit(test_fold)

#concateno matrices de training y validación
xTrainVal = np.concatenate((x_train,x_val),axis=0)
yTrainVal = np.concatenate((y_train,y_val),axis = 0)


grid=GridSearchCV(SVR( ), param, n_jobs = -1, verbose = 2)
grid.fit(xTrainVal,yTrainVal)

print('Mejor Estimator: ', grid.best_estimator_)
print('Mejores Parametros: ', grid.best_params_)

Fitting 5 folds for each of 144 candidates, totalling 720 fits
[CV] END C=1, coef0=0.01, degree=3, gamma=auto, kernel=linear; total time=   2.0s
[CV] END C=1, coef0=0.01, degree=3, gamma=auto, kernel=linear; total time=   2.4s
[CV] END C=1, coef0=0.01, degree=3, gamma=auto, kernel=linear; total time=   2.5s
[CV] END C=1, coef0=0.01, degree=3, gamma=auto, kernel=linear; total time=   2.7s
[CV] END C=1, coef0=0.01, degree=3, gamma=auto, kernel=linear; total time=   2.9s
[CV] END ..C=1, coef0=0.01, degree=3, gamma=auto, kernel=rbf; total time=   4.5s
[CV] END ..C=1, coef0=0.01, degree=3, gamma=auto, kernel=rbf; total time=   4.4s
[CV] END ..C=1, coef0=0.01, degree=3, gamma=auto, kernel=rbf; total time=   4.4s
[CV] END ..C=1, coef0=0.01, degree=3, gamma=auto, kernel=rbf; total time=   3.9s
[CV] END ..C=1, coef0=0.01, degree=3, gamma=auto, kernel=rbf; total time=   4.3s
[CV] END C=1, coef0=0.01, degree=3, gamma=auto, kernel=sigmoid; total time=   5.4s
[CV] END C=1, coef0=0.01, degree=3, gam

In [12]:
model  = grid.best_estimator_
model.fit(x_train, y_train)

SVR(C=10, coef0=0.5, gamma='auto', kernel='sigmoid')